In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly ins

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Desde los ejercicios de pandas sabemos que hay una linea con Rating 19, lo cual no es posible, la sacamos

In [ ]:
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame")

Vemos un poco como se presentan los datos del registro

In [ ]:
rdd.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

Vemos que hay apps repetidas

In [ ]:
aux = rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
aux1 = aux.filter(lambda x: x[1] > 1)
aux1.count()

798

En mi criterio las sacamos

In [ ]:
rdd1 = rdd.map(lambda x: (x[0], x)).reduceByKey(lambda x, y: x)
rdd1.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook',
  Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')),
 ('Coloring book moana',
  Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up')),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver=

Eliminamos aquellas apps cuyo tamaño varia segun el dispositivo

In [ ]:
rdd2 = rdd1.filter(lambda x: x[1].Size != "Varies with device").map(lambda x: x[1]).cache()
rdd2.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

Vemos que todos los tamaños terminan en M (megabytes) o k (kilobytes)

In [ ]:
rdd2.filter(lambda x: not x[4].endswith(("k", "M"))).count()

0

Tomamos como criterio usar todos los tamaños en kilobytes.
A aquellos que estan en megabytes les sacamos la M y los multiplicamos por mil, y a aquellos que terminan en k se la sacamos.
De paso nos quedamos unicamente con la version de android, el tamaño y un 1 que nos va a ayudar con el promedio.

In [ ]:
def convertir(tamanio):
  if tamanio.endswith("M"):
    return float(tamanio[:-1]) * 1000
  elif tamanio.endswith("k"):
    return float(tamanio[:-1])

rdd3 = rdd2.map(lambda x: (x[12], (convertir(x[4]), 1)))
rdd3.take(5)

[('4.0.3 and up', (19000.0, 1)),
 ('4.0.3 and up', (14000.0, 1)),
 ('4.0.3 and up', (8700.0, 1)),
 ('4.2 and up', (25000.0, 1)),
 ('4.4 and up', (2800.0, 1))]

Por version de android, sumamos los tamaños de las apps de esa version y los 1, para saber la cantidad de apps de esa version. Cacheamos ya que queremos verificar la cantidad de versiones así si son pocas, hacer un collect

In [ ]:
rdd4 = rdd3.reduceByKey(lambda a, b : (int(a[0]) + int(b[0]), a[1] + b[1])).cache()
rdd4.count()

34

In [ ]:
rdd4.collect()

[('4.0.3 and up', (28838254, 1359)),
 ('4.2 and up', (7466115, 353)),
 ('4.4 and up', (19557761, 774)),
 ('2.3 and up', (12279570, 606)),
 ('3.0 and up', (3171941, 231)),
 ('4.1 and up', (51019582, 2102)),
 ('4.0 and up', (20810197, 1262)),
 ('2.2 and up', (1940138, 236)),
 ('5.0 and up', (11352992, 487)),
 ('6.0 and up', (992634, 51)),
 ('1.6 and up', (361820, 116)),
 ('2.1 and up', (742732, 133)),
 ('5.1 and up', (505313, 21)),
 ('1.5 and up', (96361, 19)),
 ('7.0 and up', (972586, 40)),
 ('4.3 and up', (4647776, 212)),
 ('4.0.3 - 7.1.1', (13200, 2)),
 ('2.0 and up', (193961, 31)),
 ('2.3.3 and up', (5001637, 268)),
 ('Varies with device', (916014, 46)),
 ('3.2 and up', (354983, 36)),
 ('4.4W and up', (155400, 11)),
 ('7.1 and up', (64008, 2)),
 ('7.0 - 7.1.1', (7100.0, 1)),
 ('8.0 and up', (120500, 6)),
 ('5.0 - 8.0', (47600, 2)),
 ('3.1 and up', (134138, 10)),
 ('2.0.1 and up', (155555, 7)),
 ('4.1 - 7.1.1', (7900.0, 1)),
 ('NaN', (13100, 2)),
 ('5.0 - 6.0', (11000.0, 1)),
 ('1.0 a

Vemos que hay una version NaN, en mi criterio, la eliminamos

In [ ]:
rdd5 = rdd4.filter(lambda x: x[0] != "NaN")
rdd5.collect()

[('4.0.3 and up', (28838254, 1359)),
 ('4.2 and up', (7466115, 353)),
 ('4.4 and up', (19557761, 774)),
 ('2.3 and up', (12279570, 606)),
 ('3.0 and up', (3171941, 231)),
 ('4.1 and up', (51019582, 2102)),
 ('4.0 and up', (20810197, 1262)),
 ('2.2 and up', (1940138, 236)),
 ('5.0 and up', (11352992, 487)),
 ('6.0 and up', (992634, 51)),
 ('1.6 and up', (361820, 116)),
 ('2.1 and up', (742732, 133)),
 ('5.1 and up', (505313, 21)),
 ('1.5 and up', (96361, 19)),
 ('7.0 and up', (972586, 40)),
 ('4.3 and up', (4647776, 212)),
 ('4.0.3 - 7.1.1', (13200, 2)),
 ('2.0 and up', (193961, 31)),
 ('2.3.3 and up', (5001637, 268)),
 ('Varies with device', (916014, 46)),
 ('3.2 and up', (354983, 36)),
 ('4.4W and up', (155400, 11)),
 ('7.1 and up', (64008, 2)),
 ('7.0 - 7.1.1', (7100.0, 1)),
 ('8.0 and up', (120500, 6)),
 ('5.0 - 8.0', (47600, 2)),
 ('3.1 and up', (134138, 10)),
 ('2.0.1 and up', (155555, 7)),
 ('4.1 - 7.1.1', (7900.0, 1)),
 ('5.0 - 6.0', (11000.0, 1)),
 ('1.0 and up', (7711, 2)),
 (

Por ultimo, calculamos el promedio de tamaño de cada version de android (en kilobytes) (podemos hacer collect ya que son pocas versiones)

In [ ]:
rdd6 = rdd5.map(lambda x: (x[0], x[1][0] / x[1][1]))
rdd6.collect()

[('4.0.3 and up', 21220.201618837382),
 ('4.2 and up', 21150.467422096317),
 ('4.4 and up', 25268.425064599483),
 ('2.3 and up', 20263.31683168317),
 ('3.0 and up', 13731.34632034632),
 ('4.1 and up', 24271.92293054234),
 ('4.0 and up', 16489.85499207607),
 ('2.2 and up', 8220.92372881356),
 ('5.0 and up', 23312.098562628336),
 ('6.0 and up', 19463.41176470588),
 ('1.6 and up', 3119.137931034483),
 ('2.1 and up', 5584.451127819549),
 ('5.1 and up', 24062.52380952381),
 ('1.5 and up', 5071.631578947368),
 ('7.0 and up', 24314.65),
 ('4.3 and up', 21923.471698113208),
 ('4.0.3 - 7.1.1', 6600.0),
 ('2.0 and up', 6256.806451612903),
 ('2.3.3 and up', 18662.824626865673),
 ('Varies with device', 19913.347826086956),
 ('3.2 and up', 9860.638888888889),
 ('4.4W and up', 14127.272727272728),
 ('7.1 and up', 32004.0),
 ('7.0 - 7.1.1', 7100.0),
 ('8.0 and up', 20083.333333333332),
 ('5.0 - 8.0', 23800.0),
 ('3.1 and up', 13413.8),
 ('2.0.1 and up', 22222.14285714286),
 ('4.1 - 7.1.1', 7900.0),
 

Podemos incluso dividir los tamaños por mil para mostrar el promedio en megabytes

In [ ]:
aux = rdd6.map(lambda x: (x[0], x[1] / 1000))
aux.collect()

[('4.0.3 and up', 21.220201618837383),
 ('4.2 and up', 21.150467422096316),
 ('4.4 and up', 25.268425064599484),
 ('2.3 and up', 20.26331683168317),
 ('3.0 and up', 13.73134632034632),
 ('4.1 and up', 24.27192293054234),
 ('4.0 and up', 16.489854992076072),
 ('2.2 and up', 8.22092372881356),
 ('5.0 and up', 23.312098562628336),
 ('6.0 and up', 19.46341176470588),
 ('1.6 and up', 3.1191379310344827),
 ('2.1 and up', 5.584451127819548),
 ('5.1 and up', 24.06252380952381),
 ('1.5 and up', 5.071631578947368),
 ('7.0 and up', 24.31465),
 ('4.3 and up', 21.923471698113207),
 ('4.0.3 - 7.1.1', 6.6),
 ('2.0 and up', 6.256806451612904),
 ('2.3.3 and up', 18.662824626865675),
 ('Varies with device', 19.913347826086955),
 ('3.2 and up', 9.86063888888889),
 ('4.4W and up', 14.127272727272729),
 ('7.1 and up', 32.004),
 ('7.0 - 7.1.1', 7.1),
 ('8.0 and up', 20.083333333333332),
 ('5.0 - 8.0', 23.8),
 ('3.1 and up', 13.413799999999998),
 ('2.0.1 and up', 22.22214285714286),
 ('4.1 - 7.1.1', 7.9),
 (